In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# Any results you write to the current directory are saved as output.

similar_weather = pd.read_csv(r'../input/similar-weather-country/Similar_Weather.csv', header=None)
similar_weather_country = similar_weather.iloc[0:73,[1]]
similar_weather_country = similar_weather_country.apply(lambda x: x.str.capitalize() )

unique_similar_weather_country = similar_weather_country.drop_duplicates()
print(unique_similar_weather_country)

# %%
unique_similar_weather_country.to_csv(r"Similar_Weather_98_Percent.csv")


similar_weather = pd.read_csv(r"Similar_Weather_98_Percent.csv")
# %%
confirmed_cases = pd.read_csv(r"../input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv")
confirmed_cases_country_and_count = confirmed_cases.drop(["Province/State", "Lat", "Long"], axis = 1)
# %%
unique_countries =  confirmed_cases_country_and_count["Country/Region"].unique()

# %%
whole_country_data = confirmed_cases_country_and_count.groupby("Country/Region").sum()
whole_country_data.to_csv(r"accumulated_whole_country_data.csv")

similar_weather = pd.read_csv(r"Similar_Weather_98_Percent.csv", names=["Country/Region"], header = None)
similar_weather = similar_weather[similar_weather["Country/Region"] != "China"]
similar_weather = similar_weather[similar_weather["Country/Region"] != "Iran"]
whole_country_data = pd.read_csv(r"accumulated_whole_country_data.csv")

#%%

similar_data = pd.merge(similar_weather , whole_country_data, how='left', on = "Country/Region")
final_similar_data = similar_data.dropna()

final_similar_data.to_csv(r"weather_similar_confirmed.csv")



from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras import optimizers, Sequential
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
# %%
final_similar_data = pd.read_csv(r"weather_similar_confirmed.csv")
# %%
#sc = MinMaxScaler(feature_range=(0,1))
test_bangladesh_data = final_similar_data.iloc[[0]]
train_data = final_similar_data.iloc[1:]
# %%
TIME_STEPS = 5
LABEL_STEPS = 2
BATCH_SIZE = 36
loop_go = final_similar_data.shape[1] - TIME_STEPS - LABEL_STEPS
X_train = []
Y_train = []
for country_data in train_data.values:
    full_sequence = country_data[2:]
    for start in range(loop_go-1):
        X_train.append(full_sequence[start:TIME_STEPS+start])
        Y_train.append(full_sequence[TIME_STEPS+start: TIME_STEPS+start+LABEL_STEPS])

X_train = np.array(X_train).reshape(564, 5, 1).astype(np.float64)
Y_train = np.array(Y_train).reshape(564, 2).astype(np.float64)

# %%

#, batch_input_shape=(12, 5, 1),

model = Sequential()
model.add(LSTM(units=100,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=100,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=100,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=100))
model.add(Dropout(0.2))
model.add(Dense(units=2))
optimizer = optimizers.RMSprop(lr=0.0005)
model.compile(optimizer=optimizer, loss='mean_squared_error')
model.fit(X_train,Y_train,epochs=100,validation_split=0.25, batch_size=BATCH_SIZE)
# %%
#	Bangladesh	23.685	90.3563	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	3	3	3	3	3	3	3	5	8	10	14

x_ = np.array([ 3, 5, 8, 10, 14]).reshape(1,5, 1)
print(model.predict(x_))

test_full_sequence = test_bangladesh_data.values
# %%
test_full_sequence = test_full_sequence.reshape(test_full_sequence.shape[1], 1)[2:]
test_full_sequence = np.append(test_full_sequence, [5, 8, 10]).astype(np.float64)

#%%
X_test = []
Y_test = []
for start in range(loop_go+3-1):
    X_test.append(test_full_sequence[start:TIME_STEPS+start])
    Y_test.append(test_full_sequence[TIME_STEPS+start: TIME_STEPS+start+LABEL_STEPS])
# %%
X_test = np.array(X_test).reshape(50, 5, 1).astype(np.float64)
#Y_test = np.array(Y_test).reshape(48, 2).astype(np.float64)
predicted = np.floor(model.predict(X_test))
first_day = []
second_day = []
for pred in predicted:
    first_day.append(pred[0])
    second_day.append(pred[1])